In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas
import requests_cache
from retry_requests import retry
from geopy.geocoders import Nominatim
import openmeteo_requests

# Load dataset
file_path = "/content/drive/MyDrive/crops .csv"
data = pd.read_csv(file_path)

# Separate features and target
X = data[['temperature', 'humidity', 'ph', 'rainfall']]
y = data['label']

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.3, stratify=y_encoded, random_state=42
)

# Train Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(rf_model, X_train, y_train, cv=cv, scoring='accuracy')
rf_model.fit(X_train, y_train)

# Function to get soil pH based on location
def get_soil_ph(location):
    soil_file_path = "/content/drive/MyDrive/Soil Type.xlsx"
    df = pd.read_excel(soil_file_path)
    df.columns = df.columns.str.strip()
    df['Region'] = df['Region'].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()
    result = df[df['Region'].str.lower() == location.lower().strip()]

    if not result.empty:
        return result.iloc[0]['Soil ph']
    else:
        return None

# Function to get latitude and longitude
def get_lat_lon(location_name):
    geolocator = Nominatim(user_agent="geoapi")
    location = geolocator.geocode(location_name)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

# Function to fetch weather data
def fetch_weather_data(latitude, longitude):
    cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    openmeteo = openmeteo_requests.Client(session=retry_session)

    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": "2024-01-01",
        "end_date": "2025-01-01",
        "hourly": ["temperature_2m", "relative_humidity_2m", "rain"]
    }

    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]
    hourly = response.Hourly()

    avg_temperature = np.mean(hourly.Variables(0).ValuesAsNumpy())
    avg_humidity = np.mean(hourly.Variables(1).ValuesAsNumpy())
    avg_rain = np.mean(hourly.Variables(2).ValuesAsNumpy())

    return avg_temperature, avg_humidity, avg_rain

# Function to predict top crops
def predict_top_crops(location, top_n=10):
    lat, lon = get_lat_lon(location)
    if lat is None or lon is None:
        print("Invalid location.")
        return

    soil_ph = get_soil_ph(location)
    if soil_ph is None:
        print("Soil pH not found for location.")
        return

    temperature, humidity, rainfall = fetch_weather_data(lat, lon)

    # Prepare input sample
    sample_df = pd.DataFrame([[temperature, humidity, soil_ph, rainfall]],
                             columns=['temperature', 'humidity', 'ph', 'rainfall'])

    # Predict top crops
    probabilities = rf_model.predict_proba(sample_df)[0]
    top_indices = np.argsort(probabilities)[-top_n:][::-1]
    top_crops = [label_encoder.classes_[i] for i in top_indices]

    print("Top recommended crops:")
    print(", ".join(top_crops))

# Example usage
if __name__ == "__main__":
    location_name = input("Enter location name: ")
    predict_top_crops(location_name)


Enter location name: jaffna
Top recommended crops:
muskmelon, mungbean, mothbeans, banana, grapes, watermelon, lentil, orange, papaya, mango
